Logs   
- [2023/03/08]   
  Restart this notebook if you change the scratch library

In [36]:
import requests
import numpy as np
import tqdm
import json

from bs4 import BeautifulSoup
from typing import Iterable, List, Dict
from scratch.linear_algebra import LinearAlgebra as la
from scratch.deep_learning import DeepLearning as dl
from scratch.deep_learning import \
  Layer, Tensor, Linear, Sequential, SoftmaxCrossEntropy, Momentum

## Previous classes and functions

In [2]:
# A function to randomly choose an index based on an arbitrary set of weights
# If we give the function with weight [1, 1, 3], then on-fifth of the time
# it will return 0, one-fifth of the time it will return 1, and three fifth
# of the time it will return 2. Five is from 1 + 1 + 3 = 5.
def sample_from(weights: List[float], rng) -> int:
  """returns i with probability weights[i] / sum(weights)""" 
  total = sum(weights)
  rnd = total * rng.random()        # uniform between 0 and total
  for i, w in enumerate(weights):   
    rnd -= w                        # return the smallest i such that
    if rnd <= 0:                    # weights[0] + ... + weights[i] >= rnd
      return i

In [3]:
class Vocabulary(object):
  def __init__(self, words: List[str] = None) -> None:
    self.w2i: Dict[str, int] = {}   # mapping word -> word_id
    self.i2w: Dict[int, str] = {}   # mapping word_id -> word

    for word in (words or []):      # If words were provided
      self.add(word)                # add them.


  @property 
  def size(self) -> int:
    """how many words are in the vocabulary""" 
    return len(self.w2i)

  def add(self, word: str) -> None:
    if word not in self.w2i:        # If the word is new to us:
      word_id = len(self.w2i)       # Find the next id. 
      self.w2i[word] = word_id      # Add to the word -> word_id map. 
      self.i2w[word_id] = word      # Add to the word_id -> word map. 

  def get_id(self, word: str) -> int: 
    """return the id of the word (or None)""" 
    return self.w2i.get(word)

  def get_word(self, word_id: int) -> str:
    """return the word with the given id (or None)""" 
    return self.i2w.get(word_id)

  def one_hot_encode(self, word: str) -> Tensor:
    word_id = self.get_id(word)
    assert word_id is not None, f"unknown word {word}"

    return [1.0 if i == word_id else 0.0 for i in range(self.size)]

## Recurrent Neural Networks

This kind of neural network will capture the order of the elements in 
the input sequence. This is done by introducing *hidden state* that
maintains between inputs.

**linear layer**
```
output[o] = dot(w[o], input) + b[o]
```

**recurrent layer**
```
output[0] = dot(w[o], input) + dot(u[o], hidden) + b[o]
```

 The following layer is a simplest RNN, you should use it in practice.
 The purpose of this simplest RNN for understanding the concept behind
 recurrent neural network. More standard RNN is GRU or LSTM that can 
 be generated using TensorFlow or PyTorch library

In [4]:
class SimpleRnn(Layer):
  """Just about the simplest possible recurrent layer.""" 
  def __init__(self, input_dim: int, hidden_dim: int, rng) -> None:
    self.input_dim = input_dim
    self.hidden_dim = hidden_dim

    self.w = dl.random_tensor(hidden_dim, input_dim, init="xavier", rng=rng)
    self.u = dl.random_tensor(hidden_dim, hidden_dim, init="xavier", rng=rng)
    self.b = dl.random_normal(hidden_dim, rng=rng)

    self.reset_hidden_state()


  def reset_hidden_state(self) -> None:
    self.hidden = [0 for _ in range(self.hidden_dim)]

  
  def forward(self, input_: Tensor) -> Tensor:
    self.input = input_               # save both input and previous
    self.prev_hidden = self.hidden    # hidden state to use in backprop

    a = [(la.dot(self.w[h], input_) +         # weights @ input
          la.dot(self.u[h], self.hidden) +    # weights @ hidden
          self.b[h])                          # bias
         for h in range(self.hidden_dim)]

    self.hidden = dl.tensor_apply(dl.tanh, a)   # Apply tanh activation
    return self.hidden


  def backward(self, gradient: Tensor) -> None:
    # Backpropagate through the tanh
    a_grad = [gradient[h] * (1 - self.hidden[h] ** 2)
              for h in range(self.hidden_dim)]

    # b has the same gradient as a
    self.b_grad = a_grad

    # Each w[h][i] is multiplied by input[i] and added to a[h],
    # so each w_grad[h][i] = a_grad[h] * input[i]
    self.w_grad = [[a_grad[h] * self.input[i]
                    for i in range(self.input_dim)]
                      for h in range(self.hidden_dim)]

    # Eaach u[h][h2] is multiplied by hidden[h2] and added to a[h],
    # so each u_grad[h][h2] = a_grad[h] * prev_hidden[h2]
    self.u_grad = [[a_grad[h] * self.prev_hidden[h2]
                    for h2 in range(self.hidden_dim)]
                      for h in range(self.hidden_dim)]

    # Each input[i] is multiplied by every w[h][i] and added to a[h],
    # so each input_grad[i] = sum(a_grad[h] * w[h][i] for h in ...)
    return [sum(a_grad[h] * self.w[h][i] for h in range(self.hidden_dim))
            for i in range(self.input_dim)]

  def params(self) -> Iterable[Tensor]:
    return [self.w, self.u, self.b]

  def grads(self) -> Iterable[Tensor]:
    return [self.w_grad, self.u_grad, self.b_grad]

## Example: Using a Character-Level RNN

In this example, we want to generate an alternative brand name
from the list of top 100 brand names of successful startups

In [5]:
url = "https://www.ycombinator.com/topcompanies/"
soup = BeautifulSoup(requests.get(url).text, 'html5lib')

In [6]:
companies = [ahref.text for ahref in soup.find_all("a", "company-name")]
len(companies)

586

Build a `Vocabulary` from the characters names

In [29]:
vocab = Vocabulary([c for company in companies 
                        for c in company])

print(vocab.size)
[vocab.get_word(id_) for id_ in range(vocab.size)]

64


['S',
 'h',
 'e',
 'f',
 'Q',
 'u',
 'a',
 'n',
 't',
 's',
 'm',
 'p',
 'R',
 'i',
 'b',
 'o',
 'w',
 'M',
 ' ',
 'B',
 'r',
 'd',
 'P',
 'k',
 'H',
 'l',
 'A',
 'c',
 'L',
 'v',
 'y',
 ',',
 'I',
 '.',
 'C',
 'T',
 'K',
 'N',
 'x',
 'D',
 'g',
 'F',
 'Z',
 'G',
 'E',
 'z',
 'W',
 'j',
 'O',
 'V',
 '6',
 '4',
 'X',
 'Y',
 '1',
 'J',
 '3',
 '0',
 '-',
 'U',
 "'",
 'q',
 '9',
 '5']

We need a token to signify the start and end of the company names.
Luckily these start and stop characters do not appear in the company names

In [8]:
START = '^'
STOP = "$"

vocab.add(START)
vocab.add(STOP)

### Create the model

We'll one-hot-encode each character, passit thorugh two `SimpleRnn`s,
and then use a `Linear` layer to generate the scores for each possible next
character

In [9]:
seed = 2023_05_04
rng = np.random.default_rng(seed)

HIDDEN_DIM = 32   # You should experiment with different sizes!

rnn1 = SimpleRnn(input_dim=vocab.size, hidden_dim=HIDDEN_DIM, rng=rng)
rnn2 = SimpleRnn(input_dim=HIDDEN_DIM, hidden_dim=HIDDEN_DIM, rng=rng)
linear = Linear(input_dim=HIDDEN_DIM, output_dim=vocab.size, rng=rng)

model = Sequential([
  rnn1,
  rnn2,
  linear])

### Train the model

In [12]:
# Function to generate the company name
def generate(seed: str = START, max_len: int = 50, rng=None) -> str:
  rnn1.reset_hidden_state()     # Reset both hidden states
  rnn2.reset_hidden_state()
  output = [seed]               # Start the output with the specified seed

  # Keep going until we produce the STOP character or reach the max length
  while output[-1] != STOP and len(output) < max_len:
    # Use the last character as the input
    input_ = vocab.one_hot_encode(output[-1])

    # Generate scores using the model
    predicted = model.forward(input_)

    # Convert them to probabilities and draw a random char_id
    probabilities = dl.softmax(predicted)
    next_char_id = sample_from(probabilities, rng)

    # Add the corresponding char to our output
    output.append(vocab.get_word(next_char_id))

  # Get rid of START and STOP characters and return the word
  return ''.join(output[1:-1])

In [13]:
loss = SoftmaxCrossEntropy()
optimizer = Momentum(learning_rate=0.01, momentum=0.9)

for epoch in range(300):
  rng.shuffle(companies)      # Train in a differnt order each epoch. 
  epoch_loss = 0              # Track the loss. 

  for company in tqdm.tqdm(companies):
    rnn1.reset_hidden_state()     # Reset both hidden states.
    rnn2.reset_hidden_state() 
    company = START + company + STOP    # Add START and STOP characters

    # The rest is just our usual training loop, except that the inputs
    # and target are the one-hot-encoded previous and next characters.
    for prev, next_ in zip(company, company[1:]):
      input_ = vocab.one_hot_encode(prev)
      target = vocab.one_hot_encode(next_)
      predicted = model.forward(input_)
      epoch_loss += loss.loss(predicted, target)
      gradient = loss.gradient(predicted, target)
      model.backward(gradient)
      optimizer.step(model)

  # Each epoch, print the loss and also generate a name.
  print(epoch, epoch_loss, generate(rng=rng))

  # Turn down the learning rate for the last 100 epochs.
  # There's no principled reason for this, but it seems to work.
  if epoch == 200:
    optimizer.lr *= 0.1

100%|██████████| 586/586 [00:55<00:00, 10.55it/s]


0 17749.315081392997 CrsSdinnctii


100%|██████████| 586/586 [00:55<00:00, 10.63it/s]


1 17205.134826262944 Hayadnat S


100%|██████████| 586/586 [00:54<00:00, 10.79it/s]


...

297 6896.5990134478325 Filde


100%|██████████| 586/586 [00:52<00:00, 11.11it/s]


298 6965.857653931426 Deel


100%|██████████| 586/586 [00:52<00:00, 11.06it/s]

299 7047.8151263638665 Onbo


In [41]:
# dl.save_weights(model, "./weights/20230504_simple_rnn_weights_params.json")